# Análise de Risco de Crédito - NuBank

Risco de crédito está associado à possibilidade de um cliente não cumprir com as obrigações contratuais, como hipotecas, dívidas de cartão de crédito e outros tipos de empréstimos.

Minimizar o risco de inadimplência é uma grande preocupação para instituições financeiras. Por esse motivo, bancos comerciais e de investimento, fundos de capital de risco, empresas de gestão de ativos e seguradoras, para citar alguns, estão cada vez mais contando com a tecnologia para prever quais clientes são mais propensos a não honrar com as suas dívidas.

Modelos de Machine Learning têm ajudado essas empresas a melhorar a precisão de suas análises de risco de crédito, fornecendo um método científico para identificar devedores em potencial com antecedência.

Neste projeto, construiremos um modelo para prever o risco de inadimplência do cliente para o Nubank, uma das maiores e importantes Fintechs brasileira.

## Modelagem 

### Objetivo

- Prever o risco de inadimplência de clientes usando modelos de aprendizado de máquina, incluindo Regressão Logística,XGBoost, LightGBM e CatBoost.

- Problema: Identificar a possibilidade de clientes não cumprirem obrigações contratuais, como empréstimos e pagamentos.

### Importação das Bibliotecas

In [22]:
# Manipulação e visualização dos dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier 

# filter warnings
import warnings
warnings.filterwarnings('ignore')




### Importação do Dataset já transformado no EDA

In [23]:
df = pd.read_csv('data/acquisition_train.csv')

### Visualização dos Dados

In [24]:
df.head(2)

,ids,target_default,score_1,score_2,score_3,score_4,score_5,score_6,risk_rate,last_amount_borrowed,...,external_data_provider_fraud_score,lat_lon,marketing_channel,profile_phone_number,reported_income,shipping_state,shipping_zip_code,profile_tags,user_agent,target_fraud
0,343b7e7b-2cf8-e508-b8fd-0a0285af30aa,False,1Rk8w4Ucd5yR3KcqZzLdow==,IOVu8au3ISbo6+zmfnYwMg==,350.0,101.800832,0.259555,108.427273,0.40,25033.92,...,645,"(-29.151545708122246, -51.1386461804385)",Invite-email,514-9840782,57849.0,BR-MT,17528,"{'tags': ['n19', 'n8']}",Mozilla/5.0 (Linux; Android 6.0.1; SGP771 Buil...,NaN
1,bc2c7502-bbad-0f8c-39c3-94e881967124,False,DGCQep2AE5QRkNCshIAlFQ==,SaamrHMo23l/3TwXOWgVzw==,370.0,97.062615,0.942655,92.002546,0.24,NaN,...,243,"(-19.687710705798963, -47.94151536525154)",Radio-commercial,251-3659293,4902.0,BR-RS,40933,"{'tags': ['n6', 'n7', 'nim']}",Mozilla/5.0 (Linux; Android 5.0.2; SAMSUNG SM-...,NaN


### Exclusão das Variáveis não significativas

- Seguindo como base os processos feitos na Análise Exploratória de Dados - EDA, vamos excluir logo de início as variáveis não relevantes para o projeto.

In [25]:
# cópia do dataframe
df2 = df.copy()

In [26]:
exclude_columns = ["ids", "score_1", "score_2", "score_4", "score_5", "score_6","reason", "facebook_profile", "state", "zip", "channel", "job_name", "real_state",
                    "email", "external_data_provider_first_name", "external_data_provider_email_seen_before","lat_lon", "marketing_channel",
                    "application_time_applied", "profile_phone_number", "application_time_in_funnel","shipping_zip_code", "external_data_provider_fraud_score",
                    "profile_tags", "user_agent", "target_fraud"]

df2.drop(labels = exclude_columns, axis=1, inplace=True)

### Alteração da Variável credit_limit com valor 0

- A variável **credit_limit** possui um valor mínimo de 0.000000, isso não existe em instituições financeiras, é obrigatório liberar um valor X de crédito para o cliente. Portanto, este valor será substituído por NaN.

In [27]:
df2['credit_limit'] = df2['credit_limit'].apply(lambda x: np.nan if x == 0 else x)

### Alteração da Variável reported_income com Valor Inf

- A variável **reported_income** possui um valor máximo descrito como inf(infinito), nesse caso, vamos alterar para o tipo NaN.

In [28]:
df2.replace([np.inf, -np.inf], np.nan, inplace=True)

### Divisão dos dados em treino e teste

- Antes de iniciar as transformações e pré-processamentos dos dados, é necessário dividir o dataset em treino e teste, mas porque?
    - Porque precisamos evitar o "Data Leakage". Data Leakage ocorre quando informações do conjunto de dados de teste ou validação vazam para o conjunto de treinamento durante o pré-processamento ou modelagem, ou quando informaçõpes do target vazam para as features. Nessas situações, o que vai acontecer é que você vai ver um modelo muito bom, mas isso será ilusório, pois o seu modelo "roubou" para ter o resultado bom.

- Referências sobre Data Leakage: 
    - https://www.linkedin.com/company/universidade-dos-dados/posts/?feedView=all
    - https://www.casadocodigo.com.br/products/livro-escd
    - https://estatsite.com.br/2020/12/12/data-leakage-o-erro-que-ate-os-grandes-cometem/


In [29]:
# Dividindo os dados primeiro

X = df2.drop(columns=['target_default'])	
y = df2['target_default'].copy()                                                                                                                                                                                                                                                                                    

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Feature Engineering

Seguindo o que foi feito com as "Perguntas de Negócio utilizando o Score 3" na Análise Exploratória no eda.ipynb, que é o Score mais robusto para o projeto, vamos criar uma nova variável chamada Scores, onde vamos dividir os valores seguindo com base na descrição do Score anteriormente:

O score brasileiro é geralmente dividido em faixas:

- Baixo (0-300): Alto risco de inadimplência.
- Médio (301-700): Risco moderado.
- Alto (701-1000): Baixo risco de inadimplência.

Então vamos ter uma variável chamada de "score" que será dividida com os valores dessa forma e vamos excluir as outras colunas de score para essa nova análise.

In [30]:
def scores(score):
    if score <= 300:
        return 'baixo'
    elif score >= 301 and score <= 700:
        return 'medio'
    else:
        return 'alto'
    
X_train['score'] = X_train['score_3'].apply(scores)

In [31]:
# exclusão da coluna Score 3
exclude_columns = ["score_3"]

X_train.drop(labels = exclude_columns, axis=1, inplace=True)

### Limpeza dos Dados e Pré-Processamento

In [32]:
X_train.head().T

,25180,12555,29153,23838,35686
risk_rate,0.17,0.23,0.4,0.28,0.22
last_amount_borrowed,NaN,NaN,NaN,NaN,6029.18
last_borrowed_in_months,NaN,NaN,NaN,NaN,36.0
credit_limit,32451.0,NaN,NaN,NaN,NaN
income,95035.02,86026.89,45042.92,74039.33,55026.98
ok_since,NaN,NaN,NaN,NaN,NaN
n_bankruptcies,0.0,0.0,0.0,0.0,0.0
n_defaulted_loans,0.0,0.0,0.0,0.0,0.0
n_accounts,7.0,8.0,3.0,7.0,9.0
n_issues,7.0,NaN,NaN,NaN,NaN


### Visualizar os Valores Ausentes

In [33]:
X_train.isna().sum()

risk_rate                                             456
last_amount_borrowed                                23949
last_borrowed_in_months                             23949
credit_limit                                        15221
income                                                456
ok_since                                            21234
n_bankruptcies                                        556
n_defaulted_loans                                     464
n_accounts                                            456
n_issues                                             9209
external_data_provider_credit_checks_last_2_year    18086
external_data_provider_credit_checks_last_month         0
external_data_provider_credit_checks_last_year      12137
reported_income                                        62
shipping_state                                          0
score                                                   0
dtype: int64

In [34]:
y_train.isna().sum()

2640

### Separar as Variáveis Numéricas

In [35]:
numerical_features = X_train.select_dtypes('number').columns.to_list()
print(f'Há {len(numerical_features)} variáveis numéricas. São elas: {numerical_features}')
train_num_columns = X_train.select_dtypes(exclude='object').columns

Há 14 variáveis numéricas. São elas: ['risk_rate', 'last_amount_borrowed', 'last_borrowed_in_months', 'credit_limit', 'income', 'ok_since', 'n_bankruptcies', 'n_defaulted_loans', 'n_accounts', 'n_issues', 'external_data_provider_credit_checks_last_2_year', 'external_data_provider_credit_checks_last_month', 'external_data_provider_credit_checks_last_year', 'reported_income']


### Separar as Variáveis Categóricas

In [36]:
categorical_features = X_train.select_dtypes('object').columns.tolist()
print(f'Há {len(categorical_features)} variáveis categóricas. São elas: {categorical_features}')
train_cat_columns = X_train.select_dtypes(include='object').columns

Há 2 variáveis categóricas. São elas: ['shipping_state', 'score']


### Listar a quantidade de Subcategorias nas variáveis categóricas

In [37]:
for feature in categorical_features:
    print(feature)
    print(f"Número de Categorias: {X_train[feature].nunique()}. São elas:")
    print(X_train[feature].unique())
    print()

shipping_state
Número de Categorias: 25. São elas:
['BR-RN' 'BR-AM' 'BR-GO' 'BR-SP' 'BR-TO' 'BR-MS' 'BR-PA' 'BR-ES' 'BR-PE'
 'BR-SC' 'BR-AL' 'BR-CE' 'BR-PB' 'BR-DF' 'BR-SE' 'BR-AP' 'BR-MT' 'BR-MA'
 'BR-MG' 'BR-PR' 'BR-RO' 'BR-BA' 'BR-AC' 'BR-RR' 'BR-RS']

score
Número de Categorias: 3. São elas:
['medio' 'baixo' 'alto']



### Pré-Processamento

In [38]:
target_encoder_features = ['shipping_state']
ordinal_encoder_features = ['score']

In [39]:
# # Definindo o pipeline para transformações numéricas
# numeric_transformer = Pipeline(
#     steps=[
#         ('imputer', SimpleImputer(missing_values=np.nan,strategy='median')),
#         ('std_scaler', StandardScaler())
#     ]
# )

# # Definindo o pipeline para transformações ordinais
# ordinal_pipeline = Pipeline(
#     steps=[
#         ('imputer', SimpleImputer(strategy='most_frequent')),
#         ('ordinal_encoder', OrdinalEncoder()),
#         ('std_scaler', StandardScaler())
#     ]
# )

# # Definindo o pipeline para o target encoding
# target_pipeline = Pipeline(
#     steps=[
#         ('target_encoder', TargetEncoder(cols=target_encoder_features)),
#         ('std_scaler', StandardScaler())
#     ]
# )

# # Combinando tudo em um pré-processador
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('std_scaler', numeric_transformer, numerical_features),
#         ('ordinal', ordinal_pipeline, ordinal_encoder_features),
#         ('target', target_pipeline, target_encoder_features)
        
#     ],
#     remainder='passthrough'
# )

preprocessor = ColumnTransformer(
    transformers=[
        ('numeric_transformer', Pipeline(steps=[
            ('imputer', SimpleImputer(missing_values=np.nan,strategy='median')),
            ('scaler', StandardScaler())
        ]), numerical_features),

        ('ordinal_pipeline', Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinal_encoder', OrdinalEncoder()),
        ('std_scaler', StandardScaler())
        ]), ordinal_encoder_features),

        # ('target_pipeline', Pipeline(steps=[
        # ('target_encoder', TargetEncoder(cols=target_encoder_features)),
        # ('std_scaler', StandardScaler())
        # ]), target_encoder_features),


    ],remainder='passthrough'
)

In [40]:
X_train_prepared = preprocessor.fit_transform(X_train, y_train)
X_train_prepared.shape

(36000, 16)

In [41]:
X_train_prepared

array([[-1.2502660089590574, -0.12593559932500079, -0.29804431660898373,
        ..., -0.04017105842362287, 0.7579053378638376, 'BR-RN'],
       [-0.6578021735167316, -0.12593559932500079, -0.29804431660898373,
        ..., -0.0401710584066159, 0.7579053378638376, 'BR-AM'],
       [1.0208453602365246, -0.12593559932500079, -0.29804431660898373,
        ..., -0.04017105793488159, -1.1712887626168351, 'BR-GO'],
       ...,
       [-0.46031422836928987, 2.6020886537270584, 3.355205733756499, ...,
        -0.04017105791362191, -1.1712887626168351, 'BR-CE'],
       [0.13214960707303583, -0.12593559932500079, -0.29804431660898373,
        ..., -0.040171058014812355, -1.1712887626168351, 'BR-SC'],
       [-0.36157025579556884, -0.12593559932500079, -0.29804431660898373,
        ..., -0.0401710582339609, 0.7579053378638376, 'BR-AM']],
      dtype=object)